# <font color='red'>K-prototype algorithme for k = 2 clusters</font>

In [ ]:
import pandas as pd
from datetime import datetime
import numpy as np
import random
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

Data = pd.read_excel(r"Dataset1.xlsx")

In [160]:
data = Data.sample(1000).reset_index(drop=True)
n =len(data)
data.head()

,Num_escale,Num_Vehicule,Num_REMORQUE,Num_Client,Date_entree,Heure_entree,Date_sortie,Heure_sortie,Poids_charge,Poids_vide,Pont_entree,pont_Sortie,Produit
0,Escale11,Vehicule105,Remorque683,Client8,2020-04-25,10:17,2020-04-24,20:25,43840,16380,14,14.0,BLE TENDRE
1,Escale91,Vehicule799,Remorque920,Client12,2020-02-22,12:41,2020-02-22,12:23,43840,16460,10,10.0,CHARBON
2,Escale76,Vehicule1186,Remorque1574,Client7,2020-03-22,10:00,2020-03-22,08:03,43680,15980,19,19.0,HUILE BRUTE DE SOJA
3,Escale72,Vehicule46,Remorque46,Client1,2020-04-14,11:11,2020-04-14,09:50,43140,16020,19,19.0,MAIS
4,Escale49,Vehicule176,Remorque176,Client21,2020-05-20,13:13,2020-05-20,12:04,43500,15520,18,18.0,BLE TENDRE


# Data preprocessing

## Calculate the time difference between departure and return time

In [161]:
def time(row):
    return datetime.strptime(row, '%H:%M')

def minutes(row):
    return row.total_seconds() / (60*60)
data['date_diff_hour'] = (data['Date_entree'] - data['Date_sortie'] + data['Heure_entree'].apply(time) - data['Heure_sortie'].apply(time)).apply(minutes)

## Determine the effect of the health crisis on the shipment

In [162]:
def covid(row):
    if row < datetime.strptime('2020/03/16', '%Y/%m/%d'):
        return 'Avant_covid'
    else :
        return 'Pendant_covid'
data['Covid'] = data['Date_sortie'].apply(covid)


## Determine the effect of the day time on the shipment

In [163]:
def time(row):
    if datetime.strptime(row, '%H:%M') <= datetime.strptime('12:00', '%H:%M'):
        return 'Matin'
    elif datetime.strptime(row, '%H:%M') > datetime.strptime('12:00', '%H:%M') and datetime.strptime(row, '%H:%M') <= datetime.strptime('18:00', '%H:%M') : 
        return 'Apres-midi'
    elif datetime.strptime(row, '%H:%M') > datetime.strptime('18:00', '%H:%M'): 
        return 'Nuit'
data['Heure_entree'] = data['Heure_entree'].apply(time)
data['Heure_sortie'] = data['Heure_sortie'].apply(time)

In [164]:
data.head()

,Num_escale,Num_Vehicule,Num_REMORQUE,Num_Client,Date_entree,Heure_entree,Date_sortie,Heure_sortie,Poids_charge,Poids_vide,Pont_entree,pont_Sortie,Produit,date_diff_hour,Covid
0,Escale11,Vehicule105,Remorque683,Client8,2020-04-25,Matin,2020-04-24,Nuit,43840,16380,14,14.0,BLE TENDRE,13.866667,Pendant_covid
1,Escale91,Vehicule799,Remorque920,Client12,2020-02-22,Apres-midi,2020-02-22,Apres-midi,43840,16460,10,10.0,CHARBON,0.300000,Avant_covid
2,Escale76,Vehicule1186,Remorque1574,Client7,2020-03-22,Matin,2020-03-22,Matin,43680,15980,19,19.0,HUILE BRUTE DE SOJA,1.950000,Pendant_covid
3,Escale72,Vehicule46,Remorque46,Client1,2020-04-14,Matin,2020-04-14,Matin,43140,16020,19,19.0,MAIS,1.350000,Pendant_covid
4,Escale49,Vehicule176,Remorque176,Client21,2020-05-20,Apres-midi,2020-05-20,Apres-midi,43500,15520,18,18.0,BLE TENDRE,1.150000,Pendant_covid


# Prepare the data for the clustering

In [165]:
columns_cat = ['Num_escale', 'Num_Vehicule','Num_REMORQUE','Num_Client', 'Heure_entree', 
              'Heure_sortie', 'Covid', 'Pont_entree', 'pont_Sortie', 'Produit']
data_cat=data[columns_cat]

for c in columns_cat:
    le = LabelEncoder()
    data_cat[c] = le.fit_transform(data_cat[c])
data_cat = np.array(data_cat)

C:\Users\hamea\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [166]:
sc = MinMaxScaler()
data_num_scaled=sc.fit_transform(data[['Poids_charge', 'Poids_vide','date_diff_hour']])
data_num_array = np.array(data_num_scaled) 

## Weight factor

In [167]:
gamma = 0.5 * data_num_array.std()

# Clustering algorithm

In [168]:
def numerical_distance(data_num_i, C_num):
    dist_C1_num = np.sum((np.array(data_num_i) - np.array(C_num)[0])**2)
    dist_C2_num = np.sum((np.array(data_num_i) - np.array(C_num)[1])**2)
    return [dist_C1_num, dist_C2_num]

In [169]:
def categorical_distance(data_cat_i, C_cat):
    dist_C1_cat = np.sum(np.array(data_cat_i) != np.array(C_cat)[0])
    dist_C2_cat = np.sum(np.array(data_cat_i) != np.array(C_cat)[1])
    return [dist_C1_cat, dist_C2_cat]

In [170]:
def update_center(cluster1, cluster2, labels, data_cat):
        C_num = numerical_center(cluster1, cluster2)
        C_cat = categorical_center(labels, data_cat)
        return C_num, C_cat

In [171]:
def _labels_cost(C_num, C_cat):
    """Calculate labels and cost function given a matrix of points and
    a list of centroids for the k-prototypes algorithm.
    """
    n_points = n
    cost = 0
    labels = np.empty(n_points, dtype=np.uint16)
    for ipoint in range(n_points):
        # Numerical cost = sum of Euclidean distances
        num_costs = np.array(numerical_distance(data_num_array[ipoint], C_num))
        cat_costs = np.array(categorical_distance(data_cat[ipoint], C_cat))
        # Gamma relates the categorical cost to the numerical cost.
        tot_costs = num_costs + gamma * cat_costs
        clust = np.argmin(tot_costs)
        labels[ipoint] = clust
        cost += tot_costs[clust]

    return labels, cost

In [172]:
def categorical_center(labels, data_cat):
    C_cat_i1 = []
    C_cat_i2 = []
    for i in range(len(columns_cat)):
            List1 = np.extract(labels, data_cat[:,i]).tolist()
            C_cat_i1.append(max(set(List1), key = List1.count))
            List2 = np.extract((1 - np.array(labels)).tolist(), data_cat[:,i]).tolist()
            C_cat_i2.append(max(set(List2), key = List2.count))
    return [C_cat_i1,C_cat_i2]

In [173]:
def numerical_center(cluster1, cluster2): 
    centroids_0 = np.mean(data_num_array[cluster1], axis = 0)
    centroids_1 = np.mean(data_num_array[cluster2], axis = 0)
    return [centroids_0, centroids_1]

# Final algorithm

In [174]:
def k_prototypes(n= len(data)):
    a,b = random.sample(range(n),2)
    C_num, C_cat = data_num_array[[a,b]], data_cat[[a,b]]
    labels, cost = _labels_cost(C_num, C_cat)
    cluster1 = np.extract(labels, np.arange(n)).tolist()
    cluster2 = np.extract((1 - np.array(labels)).tolist(), np.arange(n)).tolist()    
    for itera in range(20):
        C_num,C_cat = update_center(cluster1, cluster2, labels, data_cat)
        labels, cost = _labels_cost(C_num, C_cat)
        cluster1 = np.extract(labels, np.arange(n)).tolist()
        cluster2 = np.extract((1 - np.array(labels)).tolist(), np.arange(n)).tolist()
    return len(cluster1), len(cluster2)  

## Determine the 2 clusters using k-means algorithm and numerical data only to get a preliminary idea of the problem

In [175]:
import numpy as np
from sklearn.cluster import KMeans
X = data_num_array.tolist()

kmeans = KMeans(n_clusters=2).fit(X)
labels = kmeans.labels_
cluster1 = np.extract(labels, np.arange(n)).tolist()
cluster2 = np.extract((1 - np.array(labels)).tolist(), np.arange(n)).tolist()
len(cluster1), len(cluster2)

(567, 433)

## Resulat

In [176]:
k_prototypes(n= len(data)) 

(542, 458)